# High cholesterol forecaster

## Outline


* [Part 0: Objective Summary](#Part-1:-Data-Exploration)
* [Part 1: Data Exploration](#Part-1:-Data-Exploration)
* [Part 2: Feature Preprocessing](#Part-2:-Feature-Preprocessing)
* [Part 3: Model Training and Results Evaluation](#Part-3:-Model-Training-and-Result-Evaluation)
* [Part 4: Feature Importance](#Part-4:-Feature-Selection)



# Part 0: Objective Summary
Developed a prediagnostic web app to identify people at risk of high cholesterol based on their dietary behaviors which can be used by people who are unable to visit a doctor's office due to the COVID-19 pandemic or the  expenses


# Part 1: Data Exploration

### Part 1.1: Understand the Raw Dataset

In [ ]:
#import neccessary libraries
import numpy as np
import pandas as pd
import sklearn as sl
import sklearn.preprocessing as preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns',None) 
pd.set_option('display.max_rows',None)
pd.set_option('max_colwidth',100) 


In [ ]:
#data loading 
from google.colab import files 
uploaded = files.upload()
df_origin = pd.read_csv('m_new.csv', index_col=0)
df=df_origin.copy()

In [ ]:
print ("Num of entries: " + str(df.shape[0])) 
print ("Num of features: " + str(df.shape[1])) 

### Part 1.2: Data cleaning

In [ ]:
#1 filter out the missing value for the target value
df.isnull().sum()
df1 = df[(df['Hcholes_TF']<=2)]

In [ ]:
#1 drop feautres with missing value > 20%
to_drop_list = ['edu_less16', 'edu_over20', 'drink_range', 'poor_diet_TF', 'lack_physical_activity_TF', 'extreme_hunger_TF', 'increased_tired_TF', 'very_thirsty_TF', 'love_suger_TF']
df2 = df1.drop(to_drop_list, axis=1)

In [ ]:
#2 format checking & remove duplication
df2[df2.duplicated()]
df1.info()

In [ ]:
#3 remove outlier
def iqr_outlier_rm(dt_input):
  lq,uq=np.percentile(dt_input,[25,75])
  lower_l=lq - 1.5*(uq-lq)
  upper_l=uq + 1.5*(uq-lq)
  return dt_input[(dt_input >=lower_l)&(dt_input<=upper_l)]
  
dt2_rm=iqr_outlier_rm(df2[['age', 'family_size',  'money_on_food', 'money_on_eatingout', 'money_on_deliveredfood', '#meal_eatoutside', '#meal_fastfood', '#meal_readytoeat',
 '#meal_frozenfood', '#hours_TV', 'income', 'diet_healthy_range', 'mike_like_range', 'mentalhealth_degree']])
sns.boxplot(dt2_rm,orient='v')

In [ ]:
#4 remove redundant feature pearson correlation coefficient test on all features 
all_features = list(df2.columns)
df_cor = df.drop(to_drop_list, axis=1)
corr = df_cor[all_features].corr()

fig,ax0=plt.subplots(nrows=1, figsize=(10,8))
ax0.grid(False)
ax0.set_xlabel('X')
ax0.set_ylabel('Randomization')
ax0.xaxis.label.set_size(20)
ax0.yaxis.label.set_size(20) 
sns.heatmap(corr, cmap="YlGnBu")

### Part 1.3: Data exploration

In [ ]:
#Exploratary data analysis of dietary related features
df1['Survey_year'].loc[df1['Survey_year'] == 7] = '2011-12'
df1['Survey_year'].loc[df1['Survey_year'] == 8] = '2013-14'
df1['Survey_year'].loc[df1['Survey_year'] == 9] = '2015-16'
fig,ax0=plt.subplots(nrows=1, figsize=(6,6))
sns.set_color_codes()
sns.set_style("darkgrid")
sns.barplot(x='Survey_year', y='num_meal_fastfood', data=df1)
#sns.barplot(x='Survey_year', y='num_meal_readytoeat', data=df1)
#sns.barplot(x='Survey_year', y='num_meal_frozenfood', data=df1)
#sns.barplot(x='Survey_year', y='money_on_eatingout', data=df1,  estimator=sum)
ax0.set_xlabel('')
ax0.set_ylabel('')
plt.yticks(fontsize=25)
plt.xticks(fontsize=25)

In [ ]:
#Distribution of target value on age & weight
df8['Hcholes_TF']
df_age_high = df8[(df8['Hcholes_TF']==1)]
df_age_health = df8[(df8['Hcholes_TF']==2)]
df7['Hcholes_TF']
df_weight_high = df7[(df7['Hcholes_TF']==1)]
df_weight_health = df7[(df7['Hcholes_TF']==2)]
fig,ax0=plt.subplots(nrows=1, figsize=(6,6))
ax0.xaxis.label.set_size(20) 
ax0.yaxis.label.set_size(20)
sns.set_style('darkgrid')
sns.distplot(df_age_health['age'], kde_kws={"label":"Healthy"});
sns.distplot(df_age_high['age'], kde_kws={"label":"High cholesterol"});

In [ ]:
#Distribution of target value on age & weight
fig,ax0=plt.subplots(nrows=1, figsize=(6,6))
ax0.xaxis.label.set_size(20) 
ax0.yaxis.label.set_size(20) 
sns.set_style('darkgrid')
sns.distplot(df_age_health['age'], kde_kws={"label":"Healthy"});
sns.distplot(df_age_high['age'], kde_kws={"label":"High cholesterol"});
sns.distplot(df_weight_health['weight_pounds'], kde_kws={"label":"Healthy"});
sns.distplot(df_weight_high['weight_pounds'], kde_kws={"label":"High cholesterol"});
ax0.set_xlabel('')
ax0.set_ylabel('')

# Part 2: Feature Preprocessing

In [ ]:
#Imputate missing value with mode for categorical binary features 
binary_categorical_withNan_list = ['drink_TF', 'gender','poor_TF','Hbloddpresure_TF', 'diabetes_TF','prediabetes_TF','heartfailure_TF',
 'control_weight_now_TF', 'increase_exercise_TF', 'reduce_salt_not_TF', 'vigorous_work_TF', 'moderate_work_TF', 'vigorous_recreational_activities_TF',
 'Moderate_recreational_activities_TF', 'trouble_sleep_TF', 'smoke_TF']

df4 = df3.copy()
for col in binary_categorical_withNan_list:
  mode = df4[col].mode().iloc[0]
  df4[col]=df4[col].fillna(mode)
  df4[col].loc[df[col] != 1] = 0
  df4[col] = df4[col] == 1


In [ ]:
#Imputate missing value with median for numerical features 
numerical_feature_list_8400 = ['money_onfood_range', 'money_on_eatingout', 'money_on_deliveredfood']
numerical_feature_list_5555 = ['num_meal_eatoutside', 'num_meal_fastfood', 'num_meal_readytoeat', 'num_meal_frozenfood', 'height_inches', 'weight_pounds']
df5 = df4.copy()
for col in numerical_feature_list_8400:
  median = round(df5[col].median())
  df5[col] = df5[col].fillna(median)
  df5[col].loc[df5[col] > 8400] = median  
for col in numerical_feature_list_5555:
  median = round(df5[col].median())
  df5[col] = df5[col].fillna(median)
  df5[col].loc[df5[col] > 5555] = median

In [ ]:
#One hot encoding for the rest categorical feature
df6 = df_income_final.copy()
one_hot_list =['Survey_year','diet_healthy_range', 'mike_like_range','pay_attention_nutrient_TF','mentalhealth_degree','overweight_TF']
df7 = pd.get_dummies(df6, columns= one_hot_list)

In [ ]:
#bucketing skewed feature income 
df_income['income'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
bin_income = [0, 5, 8, 14, 15]
income_range = ['0-5', '5-8', '8-14', '14-15']
df_income['income'] = pd.cut(df_income['income'], bin_income, labels= income_range)
df_income_plotonly = df_income['income'].str.get_dummies(sep=',').count()
df_income_plotonly.plot.bar(color = 'gray')

In [ ]:
#feature generation BMI
df8= df7.copy()
def BMI(series):
  weight = series["weight_pounds"]
  height = series["height_inches"]
  BMI = weight*703/height**2
  return BMI

df8["BMI"] = df8.apply(BMI,axis=1)
df8 = df8.drop(["weight_pounds", "height_inches"], axis=1)

### Part 2.2: Data scaling

In [ ]:
# Get ground truth data 
y = np.where(df8['Hcholes_TF'] == 1 ,1,0)
X = df8.drop(['Hcholes_TF'], axis=1)
#Standarized the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test)

# Part 3: Model Training

### Part 3.1: Split dataset

In [ ]:
#Under sampling for imbalanced data 
#Data splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn import model_selection


RandomUnderSampler = RandomUnderSampler()
X_resampled, y_resampled = RandomUnderSampler.fit_sample(X, y)

# Reserve 20% for testing
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_resampled, y_resampled, test_size=0.2)
print('training data has %d observation with %d features'% X_train.shape)
print('test data has %d observation with %d features'% X_test.shape)

In [ ]:
#title build models 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from xgboost import XGBClassifier

classifier_logistic = LogisticRegression()
classifier_KNN = KNeighborsClassifier()
classifier_SVC = SVC()
classifier_RF = RandomForestClassifier()
classifier_XGB = XGBClassifier()

In [ ]:
# Use 10-fold Cross Validation to get the accuracy for different models
model_names = ['Logistic Regression','Random Forest','SVM', 'KNN', 'XGB']
model_list = [classifier_logistic, classifier_KNN, classifier_RF, classifier_SVC, classifier_XGB]
count = 0

for classifier in model_list:
    cv_score = model_selection.cross_val_score(classifier, X_train, y_train, cv=10)
    print(cv_score)
    print('Model accuracy of %s is: %.3f'%(model_names[count],cv_score.mean()))
    count += 1

###Part 3.2: Model Training and Selection

###Part 3.3: Use Grid Search to Find Optimal Hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV 

# helper function for printing out grid search results 
def print_grid_search_metrics(gs):
    print ("Best score: %0.3f" % gs.best_score_)
    print ("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
# Possible hyperparamter options for Logistic Regression Regularization 
parameters = {
    'penalty':('l1', 'l2'), 
    'C':(0.01, 0.1, 1)
}
Grid_LR = GridSearchCV(LogisticRegression(),parameters, cv=10)
Grid_LR.fit(X_train, y_train)
print_grid_search_metrics(Grid_LR)
best_LR_model = Grid_LR.best_estimator_

In [ ]:
# Possible hyperparamter options for Random Forest--choose the best number of trees
parameters = {
    'n_estimators' : [50,100,200]
}
Grid_RF = GridSearchCV(RandomForestClassifier(),parameters, cv=10)
Grid_RF.fit(X_train, y_train)
print_grid_search_metrics(Grid_RF)
best_RF_model = Grid_RF.best_estimator_

In [ ]:
# Possible hyperparamter options for KNN --choose the best k
parameters = {
    'n_neighbors':[1,10,100] 
}
Grid_KNN = GridSearchCV(KNeighborsClassifier(),parameters, cv=10)
Grid_KNN.fit(X_train, y_train)
print_grid_search_metrics(Grid_KNN)
best_KNN_model = Grid_KNN.best_estimator_

In [ ]:
# Possible hyperparamter options for XGBoost 
parameters = {
    'max_depth': [5, 10, 15, 20, 25],
    'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
    'n_estimators': [50, 100, 200, 300, 500],
    'min_child_weight': [0, 2, 5, 10, 20],
    'max_delta_step': [0, 0.2, 0.6, 1, 2],
    'subsample': [0.6, 0.7, 0.8, 0.85, 0.95],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
    'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
    'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1],
    'scale_pos_weight': [0.2, 0.4, 0.6, 0.8, 1]

}
Grid_XGB = GridSearchCV(classifier_XGB,parameters, cv=10)
Grid_XGB.fit(X_train, y_train)
print_grid_search_metrics(Grid_XGB)
best_XGB_model = Grid_XGB.best_estimator_

## 4.1: Evaluate all models

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# calculate accuracy, precision and recall, [[tn, fp],[]]
def cal_evaluation(classifier, cm):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] #新家的这一句是normalized的结果了
    tn = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tp = cm[1][1]
    accuracy  = (tp + tn) / (tp + fp + fn + tn + 0.0)
    precision = tp / (tp + fp + 0.0)
    recall = tp / (tp + fn + 0.0)
    f1 = 2 * precision * recall / (precision + recall)
    print (classifier)
    print ("Accuracy is: %0.3f" % accuracy)
    print ("precision is: %0.3f" % precision)
    print ("recall is: %0.3f" % recall)
    print ("tn is: %0.3f" % tn)
    print ("fp is: %0.3f" % fp)
    print ("fn is: %0.3f" % fn)
    print ("tp is: %0.3f" % tp)
    print ("f1 score is: %0.3f" % f1)
   

# print out confusion matrices
def draw_confusion_matrices(confusion_matricies):
    class_names = ['Healthy','Hight cholesterol']
    for cm in confusion_matrices:
        classifier, cm = cm[0], cm[1]
        cal_evaluation(classifier, cm)
        fig = plt.figure()
        ax = fig.add_subplot(111)
        cax = ax.matshow(cm, interpolation='nearest',cmap=plt.get_cmap("YlGnBu"))
        #plt.title('Confusion matrix for %s' % classifier)
        fig.colorbar(cax)
        ax.set_xticklabels([''] + class_names)
        ax.set_yticklabels([''] + class_names)
        plt.xlabel('Predicted')
        plt.ylabel('Actual values')
        plt.show()

In [ ]:
#Change threshod from 0.5 to 0.3
%matplotlib inline
new =  (best_RF_model.predict_proba(X_test)[:,1]>=0.3).astype(int)
# Confusion matrix, accuracy, precison and recall for random forest and logistic regression
confusion_matrices = [
    ("Random Forest", confusion_matrix(y_test,new)),
    ("Logistic Regression", confusion_matrix(y_test,new)),
    ("KNN", confusion_matrix(y_test,best_KNN_model.predict(X_test))),
     ("XGBoost", confusion_matrix(y_test,xgb_model.predict(X_test)))
]

draw_confusion_matrices(confusion_matrices)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn import metrics

model_names = ['Logistic Regression','Random Forest','SVM', 'KNN', 'XGB']
model_list = [classifier_logistic, classifier_RF, classifier_SVC, classifier_KNN, classifier_XGB]
count = 0

for classifier in model_list:
    y_pred = classifier.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    print('Model fpr of %s is: %.3f'%(model_names[count], fpr)
    count += 1

In [ ]:
# ROC curve of all models
fig,ax0=plt.subplots(nrows=1, figsize=(6,6))
sns.set_style('darkgrid')
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf, tpr_rf, label='Random Forest')
plt.plot(fpr_xgb, tpr_xgb, label='XGBoost')
plt.plot(fpr_lr, tpr_lr, label='Logistic regression')
plt.plot(fpr_knn, tpr_knn, label='KNN')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve', fontsize=20)
plt.legend(loc='best')
plt.rc('legend',fontsize=20)
plt.xticks(rotation=0)
ax0.set_xlabel('False positive rate', fontsize=15)
ax0.set_ylabel('True positive rate',fontsize=15)
plt.show()

# Part 4: Feature Importance

### Part 4.1:  Random Forest Model - Feature Importance Discussion

In [ ]:
importances = best_RF_model.feature_importances_

# Print the feature ranking
print("Feature importance ranking by Random Forest Model:")
for k,v in sorted(zip(map(lambda x: round(x, 4), importances), X.columns), reverse=True):
    print (v + ": " + str(k))

In [ ]:
#Feature importancefor young people 
df8.columns = df8_col_importance
df_young = df8[(df8['age']<=24)]
y_young = np.where(df_young['Hcholes_TF'] == 1 ,1,0)
X_young = df_young.drop(['Hcholes_TF'], axis=1)
forest = RandomForestClassifier()
forest.fit(X_young, y_young)
importances_young = forest.feature_importances_
df_importance_young = pd.DataFrame(importances_young, index=X_young.columns.values, columns=['feature importance']).sort_values('feature importance', ascending=False)
df_young_importance = df_importance_young[(df_importance_young['feature importance']> 0.043)].reset_index() 
fig,ax0=plt.subplots(nrows=1, figsize=(6,6))
sns.set_style('darkgrid')
sns.set_color_codes()
certain = ['gray' if (x != df_young_importance.iloc[4][1]) else 'red' for x in df_young_importance['feature importance'] ]
sns.barplot(y = 'index' , x='feature importance', data=df_young_importance, palette=certain)
ax0.set_xlabel('')
ax0.set_ylabel('')
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)